In [2]:
from rateslib import *

In [ ]:
curve = Curve({
    dt(2022, 12, 1): 1.0,
    dt(2022, 12, 31): 1.0,
    dt(2023, 1, 1): 1.0,
    dt(2023, 2, 1): 1.0,
}, "log_linear")
instruments = [
    IRS(dt(2022, 12, 1), "1d", "A", curves=curve),
    Spread(
        IRS(dt(2022, 12, 30), "1d", "A", curves=curve),
        IRS(dt(2022, 12, 31), "1d", "A", curves=curve),
    ),
    IRS(dt(2023, 1, 1), "1d", "A", curves=curve),
]
turn_solver = Solver(
    curves=[curve], 
    instruments=instruments, 
    s=[0.0, -0.5, 0.0],
    instrument_labels=["null1", "turn", "null2"],
)

In [ ]:
curve.plot("1d")

In [ ]:
linecurve = LineCurve({
    dt(2022, 12, 1): 0.0,
    dt(2022, 12, 31): -50.0,
    dt(2023, 1, 1): 0.0,
}, "flat_forward")
instruments = [
    Value(dt(2022, 12, 1), curves=linecurve),
    Value(dt(2022, 12, 31), curves=linecurve),
    Value(dt(2023, 1, 1), curves=linecurve),
]
solver = Solver(curves=[linecurve], instruments=instruments, s=[0.0, -0.5, 0.0])

In [ ]:
linecurve.plot("1d", right=dt(2023, 2, 1))

In [ ]:
cubiccurve = Curve({
    dt(2022, 12, 1): 1.0,
    dt(2022, 12, 21): 1.0,
    dt(2023, 1, 11): 1.0,
    dt(2023, 2, 1): 1.0,
}, "log_linear", t = [
    dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1),
    dt(2022, 12, 21),
    dt(2023, 1, 11),
    dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1),
])
instruments = [
    IRS(dt(2022, 12, 1), "20d", "A", curves=cubiccurve),
    IRS(dt(2022, 12, 21), "20d", "A", curves=cubiccurve),
    IRS(dt(2023, 1, 11), "20d", "A", curves=cubiccurve),
]
solver = Solver(curves=[cubiccurve], instruments=instruments, s=[2.10, 2.20, 2.35])

In [ ]:
cubiccurve.plot("1b")

In [ ]:
compcurve = CompositeCurve([cubiccurve, curve])

In [ ]:
print(IRS(dt(2022, 12, 21), "20d", "A").rate(curves=compcurve))

In [ ]:
cubiccurve = Curve({
    dt(2022, 12, 1): 1.0,
    dt(2022, 12, 21): 1.0,
    dt(2023, 1, 11): 1.0,
    dt(2023, 2, 1): 1.0,
}, "log_linear", t = [
    dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1),
    dt(2022, 12, 21),
    dt(2023, 1, 11),
    dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1),
])
compositecurve = CompositeCurve([curve, cubiccurve])
instruments = [
    IRS(dt(2022, 12, 1), "20d", "A", curves=compositecurve),
    IRS(dt(2022, 12, 21), "20d", "A", curves=compositecurve),
    IRS(dt(2023, 1, 11), "20d", "A", curves=compositecurve),
]
solver = Solver(
    curves=[cubiccurve, compositecurve], 
    instruments=instruments, 
    s=[2.10, 2.20, 2.35],
    instrument_labels=["irs1", "irs2", "irs3"],
    pre_solvers=[turn_solver],
)

In [ ]:
compositecurve.plot("1d", comparators=[cubiccurve])

In [ ]:
irs = IRS(dt(2022, 12, 31), dt(2023, 1, 1), "A", notional=1e9)
irs.delta(curves=compositecurve, solver=solver)

In [ ]:
turn_curve = Curve({
    dt(2022, 12, 1): 1.0,
    dt(2022, 12, 31): 1.0,
    dt(2023, 1, 1): 1.0,
    dt(2023, 2, 1): 1.0,
}, "log_linear")
cubic_curve = Curve({
    dt(2022, 12, 1): 1.0,
    dt(2022, 12, 21): 1.0,
    dt(2023, 1, 11): 1.0,
    dt(2023, 2, 1): 1.0,
}, t = [
    dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1), dt(2022, 12, 1),
    dt(2022, 12, 21),
    dt(2023, 1, 11),
    dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1), dt(2023, 2, 1),
])
composite_curve = CompositeCurve([turn_curve, cubic_curve])
instruments = [
    IRS(dt(2022, 12, 1), "1d", "A", curves=turn_curve),
    Spread(
        IRS(dt(2022, 12, 30), "1d", "A", curves=turn_curve),
        IRS(dt(2022, 12, 31), "1d", "A", curves=turn_curve),
    ),
    IRS(dt(2023, 1, 1), "1d", "A", curves=turn_curve),
    IRS(dt(2022, 12, 1), "20d", "A", curves=composite_curve),
    IRS(dt(2022, 12, 21), "20d", "A", curves=composite_curve),
    IRS(dt(2023, 1, 11), "20d", "A", curves=composite_curve),
]
solver = Solver(
    curves=[turn_curve, cubic_curve, composite_curve], 
    instruments=instruments, 
    s=[0.0, -0.5, 0.0, 2.01, 2.175, 2.35],
    instrument_labels=["zero1", "turn", "zero2", "irs1", "irs2", "irs3"],
)

In [ ]:
f, a, l = composite_curve.plot("1d")

In [ ]:
turn_curve = LineCurve({
    dt(2022, 9, 15): 0.0,
    dt(2022, 10, 1): 0.0,
    dt(2023, 1, 1): 0.0,
}, "flat_forward")
line_curve = LineCurve({
    dt(2022, 9, 15): 1.0,
    dt(2022, 12, 15): 1.0,
    dt(2023, 3, 15): 1.0,
}, "linear")
composite_curve=CompositeCurve([turn_curve, line_curve])
instruments = [
    Value(dt(2022, 9, 15), curves=turn_curve),
    Value(dt(2022, 10, 1), curves=turn_curve),
    Value(dt(2023, 1, 1), curves=turn_curve),
    Value(dt(2022, 9, 15), curves=composite_curve),
    Value(dt(2022, 12, 15), curves=composite_curve),
    Value(dt(2023, 3, 15), curves=composite_curve),
]
solver = Solver(
    curves=[turn_curve, line_curve, composite_curve], 
    instruments=instruments, 
    s=[0.0, -0.2, 0.0, 3.5, 3.7, 4.05],
    instrument_labels=["zero1", "turn", "zero2", "fra1", "fra2", "fra3"],
)

In [ ]:
composite_curve.plot("3m", right=dt(2023, 3, 15))

In [ ]:
turn_curve = LineCurve({
    dt(2022, 9, 15): 0.0,
    dt(2022, 9, 30): 0.0,
    dt(2022, 10, 1): 0.0,
    dt(2022, 12, 31): 0.0,
    dt(2023, 1, 1): 0.0,
    dt(2023, 3, 15): 0.0,
}, "linear")
line_curve = LineCurve({
    dt(2022, 9, 15): 1.0,
    dt(2022, 12, 15): 1.0,
    dt(2023, 3, 15): 1.0,
}, "linear")
composite_curve=CompositeCurve([turn_curve, line_curve])
instruments = [
    Value(dt(2022, 9, 15), curves=turn_curve),
    Value(dt(2022, 9, 30), curves=turn_curve),
    Value(dt(2022, 10, 1), curves=turn_curve),
    Value(dt(2022, 12, 31), curves=turn_curve),
    Value(dt(2023, 1, 1), curves=turn_curve),
    Value(dt(2023, 3, 15), curves=turn_curve),
    Value(dt(2022, 9, 15), curves=composite_curve),
    Value(dt(2022, 12, 15), curves=composite_curve),
    Value(dt(2023, 3, 15), curves=composite_curve),
]
solver = Solver(
    curves=[turn_curve, line_curve, composite_curve], 
    instruments=instruments, 
    s=[0.0, 0.0, -0.2, -0.04, 0.0, 0.0, 3.5, 3.7, 4.05],
    instrument_labels=["zero1", "zero2", "turnA", "turnB", "zero3", "zero4", "fra1", "fra2", "fra3"],
)

In [ ]:
composite_curve.plot("3m", right=dt(2023, 3, 15))

In [3]:
zfl = ZeroFixedLeg(
    dt(2022, 1, 1), "30y", "A", calendar="ldn", fixed_rate=2.5, notional=100e6, convention="1+",
)

In [4]:
zfl.cashflows()

,Type,Period,Ccy,Acc Start,Acc End,Payment,Convention,DCF,Notional,DF,Rate,Spread,Cashflow,NPV,FX Rate,NPV Ccy
0,ZeroFixedLeg,None,USD,2022-01-01,2052-01-01,2052-01-04,1+,30.0,100000000.0,None,2.5,None,-1.097568e+08,None,1.0,None


In [5]:
zfl.periods[0].fixed_rate

3.658558596939286

In [6]:
zfl.periods[0].cashflows()

{'Type': 'FixedPeriod',
 'Period': 'Regular',
 'Ccy': 'USD',
 'Acc Start': datetime.datetime(2022, 1, 1, 0, 0),
 'Acc End': datetime.datetime(2052, 1, 1, 0, 0),
 'Payment': datetime.datetime(2052, 1, 4, 0, 0),
 'Convention': '1+',
 'DCF': 30.0,
 'Notional': 100000000.0,
 'DF': None,
 'Rate': 3.658558596939286,
 'Spread': None,
 'Cashflow': -109756757.90817858,
 'NPV': None,
 'FX Rate': 1.0,
 'NPV Ccy': None}